### First to the current working directory

In [100]:
import os
os.chdir(r"C:\Users\Amreet\Desktop\Chicken Disease Classification")
%pwd

'C:\\Users\\Amreet\\Desktop\\Chicken Disease Classification'

In [101]:
import os
import shutil
import zipfile
import yaml
from pathlib import Path
from box import ConfigBox
from dataclasses import dataclass
from dataclasses import asdict
import logging

### Update the Entity

In [102]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class DataIngestionConfig:
    root_dir: str
    source_URL: str
    local_data_file: str
    unzip_dir: str

In [103]:
# ---- Logger ----
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

### Update the Configuration Manager in src config

In [104]:
from chicken_classifier.constants import *
from chicken_classifier.utils.common import *

In [105]:
class ConfigurationManager:
    def __init__(self, config_filepath="config.yaml"):
        with open(config_filepath, "r") as f:
            self.config = ConfigBox(yaml.safe_load(f))

        os.makedirs(self.config.artifacts_root, exist_ok=True)

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        cfg = self.config.data_ingestion
        os.makedirs(cfg.root_dir, exist_ok=True)

        return DataIngestionConfig(
            root_dir=cfg.root_dir,
            source_URL=cfg.source_URL,
            local_data_file=cfg.local_data_file,
            unzip_dir=cfg.unzip_dir
        )

### Update the components

In [106]:
import os
import urllib.request as request
import zipfile
import shutil
from chicken_classifier import logger

In [107]:
import os
import shutil
import zipfile
from pathlib import Path
import logging

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        local_file = Path(self.config.local_data_file)

        if not local_file.exists():
            shutil.copy(self.config.source_URL, local_file)
            logger.info(f"Copied local file from {self.config.source_URL} → {local_file}")
        else:
            logger.info(f"File already exists: {local_file} ({get_size(local_file)})")

    def extract_zip_file(self):
        unzip_path = Path(self.config.unzip_dir)
        os.makedirs(unzip_path, exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

        logger.info(f"Extracted {self.config.local_data_file} → {unzip_path}")


### Update Pipeline

In [108]:
try:
    config = ConfigurationManager("config/config.yaml")
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)

    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e


[2025-09-19 00:04:15,542: INFO: 1031855767: Copied local file from C:/Users/Amreet/Desktop/New folder (6)/chicken_image.zip → artifacts\data_ingestion\data.zip]
[2025-09-19 00:04:28,066: INFO: 1031855767: Extracted artifacts/data_ingestion/data.zip → artifacts\data_ingestion]
